## Часть 1. Пример проблемы использования метода Гаусса для решения СЛАУ

Перед вами простая реализация метода Гаусса для решения СЛАУ. Далее по коду представлены две СЛАУ $A_1x = b_1$ и $A_2x = b_2$, эквивалентные с точностью до перестановки строк. Эти СЛАУ решаются сначала пакетным методом, затем реализованным методом Гаусса. Для пакетного метода получается два одинаковых решения. Для метода Гаусса - два отличающихся решения. ЗАДАНИЕ: необходимо объяснить, почему для представленного метода Гаусса решения различаются.

In [9]:
import numpy as np
import numpy.linalg as la
import copy 

def gauss( A_in, b_in ):
    n = b_in.size
    A = copy.deepcopy(A_in)
    b = copy.deepcopy(b_in)
    for k in range(0,n-1):
        for i in range(k+1,n):
            if A[i,k]!=0:
                c = A[i,k]/A[k,k]
                A[i,k+1:n] = A[i,k+1:n] - c*A[k,k+1:n]
                b[i] = b[i] - c*b[k]
                
    # обратный ход
    for k in range(n-1,-1,-1):
        b[k] = (b[k] - np.dot(A[k,k+1:n],b[k+1:n]))/A[k,k]
    return b

In [10]:
A1 = np.array([[1e-16, 1., -1.], [-1., 2., -1.], [2., -1., 0.]])
b1 = np.array([0., 0., 1.])

A2 = np.array([[2., -1., 0.], [-1., 2., -1.], [1e-16, 1., -1.]])
b2 = np.array([1., 0., 0.])

print('mu1 = ', la.cond(A1))
print('mu2 = ', la.cond(A2))

mu1 =  16.39373162228438
mu2 =  16.393731622284395


In [11]:
print('u1 = ', la.solve(A1, b1))
print('u2 = ', la.solve(A2, b2))

u1 =  [1. 1. 1.]
u2 =  [1. 1. 1.]


In [12]:
print('u1 = ', gauss(A1, b1))
print('u2 = ', gauss(A2, b2))

u1 =  [0.55511151 0.25       0.25      ]
u2 =  [1. 1. 1.]


Заметим, что для первой матрицы системы A для 1ой строки не выполнено условие диагонального преобладания (a_11 << a_12 + a_13). Из-за чего при выполнение метода Гаусса после деления на диагональный элемент возникает ошибка.

## Часть 2. LU разложение

Задание: 

реализовать алгоритм решения предыдущей задачи с матрицей A2 с помощью LU-разложение В решении должна выводиться L, U и собственно решение системы. 

ВАЖНО: реализация метода LU должна быть получена путем небольшой модификации метода gauss!  При это саму реализацию можно разделить на два метода: один метод собственно находит LU разложение (можно сделать переделкой цикла для матрицы A метода gauss), второй метод - непосредственное решение системы с помощью прямого и обратного хода. Ни в каком виде нельзя пользоваться пакетными методами (в частности, la.solve)

Заметим, что при прямом ходе Гаусса мы получаем верхнетреугольную матрицу U = A_n-1, а коэффициенты нижнетреугольной матрицы L получаются как коэффициенты c = a_ik/a_kk. Доказывается в 2.4 Лобанов, Петров.

In [5]:
import numpy as np
import numpy.linalg as la
import copy 

def LU_decomposition(A_in):
    A = copy.deepcopy(A_in)
    n = A.shape[0]
    L = np.eye(n)

    for k in range(0,n-1):
        for i in range(k+1,n):
            if A[i,k]!=0:
                L[i, k] = A[i,k]/A[k,k] # Записываем коэффициент в L
                A[i] = A[i] - L[i, k] * A[k]
    
    return L, A

A = np.array([[2., -1., 0.], [-1., 2., -1.], [1e-16, 1., -1.]])
b = np.array([1., 0., 0.])

def solve_lu(A, b):
    L, U = LU_decomposition(A)
    n = L.shape[0]

    #print(L, '\n\n\n')
    #print(U, '\n\n\n')
    
    # Решение Ly = b (нижнетреугольная матрица) прямой ход
    y = np.zeros(n)
    for i in range(n):
        y[i] = b[i] - np.dot(L[i, :i], y[:i])
    
    # Решение Ux = y (верхнереугольная матрица) обратный ход
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]
        
    return x

solution = solve_lu(A, b)
print(solution)

[1. 1. 1.]


### LU - разложение с помощью пакета sympy

Чтобы убедиться, что разложение получено верно, можно воспользоваться скриптом ниже

In [6]:
import sympy as sp

A = sp.Matrix([[2., -1., 0.], [-1., 2., -1.], [0, 1., -1.]])
L, U, _ = A.LUdecomposition()
L

Matrix([
[   1,                 0, 0],
[-0.5,                 1, 0],
[   0, 0.666666666666667, 1]])

In [7]:
U

Matrix([
[2.0, -1.0,                0.0],
[  0,  1.5,               -1.0],
[  0,    0, -0.333333333333333]])

## Часть 3. Нахождение обратной матрицы с помощью LU разложения

Задание:

Предложить алгоритм с использованием LU-разложения и найти обратную матрицу с точностью $\epsilon = 10^{-3}$:
$$
A = \begin{pmatrix} 
1 & 1  & 1 \\
0 & 1 & 2 \\
7 & 1 & 4 \\
\end{pmatrix}
$$
Для необходимых оценок использовать первую норму. Сравнить результат со значением, найденным с помощью функции numpy.linalg.inv.

Решение: для уравнения AX = E требуется найти матрицу X. Рассмотрим i-ый столбец первого уравнения, тогда Ax = e_i (единичный вектор в iой строке), используя LU-разложение получим следующие уравнения Ly = e_i, Ux = y, которые мы умеем решать. Поочередно решив эти уравнения получим все столбцы интересующей нас матрицы X (A^-1).

In [8]:
import numpy as np

A = np.array([[1., 1., 1.], [0., 1., 2.], [7., 1., 4.]])

def LU_decomposition(A_in):
    A = copy.deepcopy(A_in)
    n = A.shape[0]
    L = np.eye(n)

    for k in range(0,n-1):
        for i in range(k+1,n):
            if A[i,k]!=0:
                c = A[i,k]/A[k,k]
                L[i, k] = c  # Записываем коэффициент в L
                A[i] = A[i] - c * A[k]
    
    return L, A

def inverse_matrix(A):
    n = A.shape[0]
    L, U = LU_decomposition(A)
    inv_A = np.zeros((n,n))
    
    for i in range(n):
        e_i = np.zeros(n)
        e_i[i] = 1  # единичный вектор

        # Решение Ly = e_i (нижнетреугольная матрица) прямой ход
        y = np.zeros(n)
        for j in range(n):
            y[j] = e_i[j] - np.sum(L[j, :j] * y[:j])

        # Решение Ux = y (верхнереугольная матрица) обратный ход
        x = np.zeros(n)
        for j in range(n - 1, -1, -1):
            x[j] = (y[j] - np.sum(U[j, j + 1:] * x[j + 1:])) / U[j, j]

        inv_A[:, i] = x

    return inv_A

A = np.array([[1., 1., 1.], [0., 1., 2.], [7., 1., 4.]])

inv_A_lu = inverse_matrix(A)
inv_A_np = np.linalg.inv(A)

norm_difference = np.linalg.norm(inv_A_lu - inv_A_np, ord=1)

print(inv_A_lu, "\n\n")
print(inv_A_np, "\n\n")

print(norm_difference)

[[ 0.22222222 -0.33333333  0.11111111]
 [ 1.55555556 -0.33333333 -0.22222222]
 [-0.77777778  0.66666667  0.11111111]] 


[[ 0.22222222 -0.33333333  0.11111111]
 [ 1.55555556 -0.33333333 -0.22222222]
 [-0.77777778  0.66666667  0.11111111]] 


7.216449660063518e-16


## Часть 4. Модифицированный метод Гаусса

Модифицировать метод Гаусса из Части 1 так, чтобы система $A_1x = b_1$ решалась корректно. ВАЖНО: реализация метода должна быть получена путем модификации метода gauss, а не переписыванием кода с нуля! 

In [9]:
import copy 

def gauss_main_elem( A_in, b_in ):
    n = b_in.size
    A = copy.deepcopy(A_in)
    b = copy.deepcopy(b_in)
    for k in range(0,n-1):

        # Поиск максимального элемента в столбце
        max_index = np.argmax(np.abs(A[k:, k])) + k
        # Обмен строк
        A[[k, max_index]] = A[[max_index, k]]
        b[k], b[max_index] = b[max_index], b[k]

        for i in range(k+1,n):
            if A[i,k]!=0:
                c = A[i,k]/A[k,k]
                A[i,k+1:n] = A[i,k+1:n] - c*A[k,k+1:n]
                b[i] = b[i] - c*b[k]
                
    # обратный ход
    for k in range(n-1,-1,-1):
        b[k] = (b[k] - np.dot(A[k,k+1:n],b[k+1:n]))/A[k,k]
    return b

A1 = np.array([[1e-16, 1., -1.], [-1., 2., -1.], [2., -1., 0.]])
b1 = np.array([0., 0., 1.])

A2 = np.array([[2., -1., 0.], [-1., 2., -1.], [1e-16, 1., -1.]])
b2 = np.array([1., 0., 0.])

print('u1 = ', gauss_main_elem(A1, b1))
print('u2 = ', gauss_main_elem(A2, b2))

u1 =  [1. 1. 1.]
u2 =  [1. 1. 1.]
